## Colab 환경에서 환경설정
- 앞으로의 남은 실습은 Colab에서 실행한다는 가정하에 실습을 진행합니다.

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# findspark를 사용해 Spark 환경 초기화
import findspark

# findspark는 Python 환경에서 PySpark를 사용할 수 있도록 Spark 경로를 자동으로 설정해주는 라이브러리입니다.
# findspark.init()은 환경 변수를 설정해 PySpark가 올바르게 작동하도록 합니다.
# init() 메서드에 경로를 지정하지 않으면, 기본적으로 SPARK_HOME 환경 변수를 자동으로 검색합니다.
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

## 라이브러리 및 spark 시작 및 처리를 위한 객체 생성

In [3]:
# PySpark 라이브러리를 임포트합니다. PySpark는 대규모 데이터를 병렬로 처리하고 분석하는 데 사용됩니다.
import pyspark

# DataFrame 및 SQL 작업을 수행하기 위해 SparkSession을 임포트합니다.
# SparkSession은 PySpark에서 DataFrame 및 SQL 작업을 위한 진입점 역할을 합니다.
from pyspark.sql import SparkSession

# Spark 설정을 구성하고 SparkContext를 생성하기 위해 필요한 모듈을 임포트합니다.
# SparkConf는 Spark 애플리케이션의 설정을 정의하는 데 사용되고,
# SparkContext는 RDD API를 사용하여 데이터 처리를 수행하기 위한 기본 객체입니다.
from pyspark import SparkConf, SparkContext

# 새로운 SparkSession을 생성합니다. 'transformations'은 애플리케이션 이름으로,
# Spark UI 및 로그에 표시됩니다. SparkSession은 DataFrame과 SQL 작업을 수행하는 데 사용됩니다.
spark = SparkSession.builder.appName('transformations').getOrCreate()

# SparkContext 객체를 가져옵니다. SparkContext는 RDD API 작업을 수행할 때 사용되는 기본 객체입니다.
# RDD(Resilient Distributed Dataset)는 PySpark의 저수준 API로, 데이터를 분산 처리하기 위해 사용됩니다.
sc = spark.sparkContext

## 1. Map 변환
- 승객 데이터에서 Survived 여부와 승객의 나이를 가져옵니다.

In [4]:
# 타이타닉 데이터 RDD 생성: ("이름", 생존 여부, 나이) 형태의 데이터를 포함
# 데이터는 2개의 파티션으로 나뉘어 분산 처리됨
titanic_data = sc.parallelize([("Jack", 0, 25), ("Rose", 1, 22), ("John", 1, 30)], 2)

# RDD의 파티션 수를 가져옴
num_partitions = titanic_data.getNumPartitions()

# 각 파티션에 포함된 데이터를 확인
print(titanic_data.glom().collect())
print(num_partitions)

# 출력 결과: 각 파티션의 데이터와 파티션 수

[[('Jack', 0, 25)], [('Rose', 1, 22), ('John', 1, 30)]]
2


In [5]:
# TODO: 위의 코드를 따라 치면서 python을 활용한 spark 사용 방법에 익숙해지세요!


## 2. Filter 변환
- 생존한 승객만 필터링합니다.

In [7]:
# 생존 여부가 1(생존자)인 데이터만 필터링
survived_filter = titanic_data.filter(lambda x: x[1] == 1)
print(survived_filter.collect())

# 출력 결과: 생존자 데이터만 포함된 리스트

[('Rose', 1, 22), ('John', 1, 30)]


In [8]:
# TODO: 위의 코드를 따라 치면서 python을 활용한 spark 사용 방법에 익숙해지세요!


## 3. GroupBy 변환
- 이름의 첫 글자로 승객 그룹화

In [9]:
# 타이타닉 데이터 RDD를 이름의 첫 글자를 기준으로 그룹화
grouped_by_initial = titanic_data.groupBy(lambda x: x[0][0])

# 각 그룹의 키(이름의 첫 글자)와 해당 그룹에 속하는 데이터를 리스트로 변환하여 출력
print([(t[0], list(t[1])) for t in grouped_by_initial.collect()])

# 출력 결과: 이름의 첫 글자를 기준으로 그룹화된 데이터

[('J', [('Jack', 0, 25), ('John', 1, 30)]), ('R', [('Rose', 1, 22)])]


In [10]:
# TODO: 위의 코드를 따라 치면서 python을 활용한 spark 사용 방법에 익숙해지세요!


## 4. GroupByKey 변환
- 객실 등급별로 승객의 나이 정보 그룹화

In [11]:
# 승객 데이터를 (클래스, 나이) 형식으로 생성
passengers = sc.parallelize([(1, 25), (2, 30), (1, 22), (3, 45)])

# 클래스별로 승객의 나이를 그룹화
grouped_by_class = passengers.groupByKey()

# 각 클래스와 해당 클래스의 나이 데이터를 리스트로 변환하여 출력
print([(t[0], list(t[1])) for t in grouped_by_class.collect()])

# 출력 결과: 클래스별로 그룹화된 나이 데이터

[(1, [25, 22]), (2, [30]), (3, [45])]


In [12]:
# TODO: 위의 코드를 따라 치면서 python을 활용한 spark 사용 방법에 익숙해지세요!


## 5. Word Counting Using ReduceByKey 변환
- Survived 값으로 생존자와 비생존자 수 계산

In [13]:
# 생존 여부를 키로, 값은 1로 매핑하여 생성 (0: 사망, 1: 생존)
survived_counts = titanic_data.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)

# 생존 여부별 승객 수를 출력
print(survived_counts.collect())

# 출력 결과: 생존 여부별 승객 수


[(0, 1), (1, 2)]


In [14]:
# TODO: 위의 코드를 따라 치면서 python을 활용한 spark 사용 방법에 익숙해지세요!


## 6. Join 변환
- 승객 이름과 객실 등급을 결합하여 join

In [17]:
# 승객 이름과 클래스 정보를 포함한 RDD 생성
passengers_info = sc.parallelize([("Jack", 1), ("Rose", 2)])

# 승객 이름과 생존 여부 정보를 포함한 RDD 생성
survival_info = sc.parallelize([("Jack", 0), ("Rose", 1)])

# 승객 이름을 기준으로 두 RDD를 조인
joined_data = passengers_info.join(survival_info)

# 조인된 데이터를 출력
print(joined_data.collect())

# 출력 결과: 승객 이름을 기준으로 병합된 데이터 (이름, (클래스, 생존 여부))

[('Rose', (2, 1)), ('Jack', (1, 0))]


In [16]:
# TODO: 위의 코드를 따라 치면서 python을 활용한 spark 사용 방법에 익숙해지세요!
